In [ ]:
import pandas as pd
import json
smalltalkintent = pd.read_csv("intentdata/Small_talk_Intent.csv").rename(columns={"Utterances":"text","Intent":"intent"})
training = pd.read_csv("intentdata/train_command.csv")
df3 = pd.concat([training,smalltalkintent], ignore_index=True)
df3.to_csv("intentdata/train.csv",mode="w",index=False)
# TODO Do NLPAugmentation
#df3 = pd.concat([training,smalltalkintent], ignore_index=True)
#with open("intentdata/Intent.json","r") as f:
#    chatbotXresponseintent = json.load(f)
#chatbotXresponseintent["intents"][0] 


In [ ]:
# Produce responses
import json 
with open("intentdata/Intent.json","r") as f:
    data = json.load(f)
intents = []
for intent in data["intents"]:
    intents.append({"intent":intent["intent"],"responses":intent["responses"]})
with open("intentdata/responses.json","w+") as f:
    json.dump({"response":intents},f)
    

In [ ]:
with open("intentdata/responses.json","r") as f:
  responses = json.load(f)["responses"]
#print(intents[0] in greetings)
#if intents[0] in greetings:
print(responses["Greeting"])

In [ ]:
import json
import pandas as pd 
from IPython.display import display
with open("intentdata/aug/intent_aug_text.json","r") as f:
    data = json.load(f)["intents"]
columns_to_concat = []
for i in range(len(data)):
    column1 = pd.DataFrame.from_dict({"text":data[i]["text"]})
    #print(column1)
    column2 = pd.DataFrame.from_dict({"intent":[data[i]["intent"] for j in range(len(data[i]["text"]))]})
    #print(column2)
    tent_concat= pd.concat([column1,column2],axis=1)
    #print(tent_concat)
    columns_to_concat.append(tent_concat)
intentdf = pd.concat(columns_to_concat,axis=0)
intentdf
df = pd.read_csv("intentdata/train.csv")
newdf = pd.concat([df,intentdf],axis=0)
newdf.to_csv("intentdata/train.csv",mode="w",index=False)




In [ ]:
import pandas as pd
data = pd.read_csv("intentdata/train.csv")


In [ ]:
import pandas as pd
data = pd.read_csv("intentdata/train_no_half_response.csv") # Original data is a (2000, 7) DataFrame
data = data.replace("smalltalk_agent_acquaintance","CourtesyGreeting")
data = data.replace("smalltalk_agent_age","CurrentHumanQuery")
data = data.replace("smalltalk_agent_annoying","NotTalking2U")
data = data.replace("smalltalk_agent_bad","Swearing")
data = data.replace("smalltalk_agent_boss","CurrentHumanQuery")
data = data.replace("smalltalk_agent_clever","Clever")

data = data.replace("smalltalk_agent_beautiful","Clever")
data = data.replace("smalltalk_agent_fired","Shutup")
data = data.replace("smalltalk_agent_good","Thanks")
data = data.replace("smalltalk_agent_chatbot","SelfAware")
data = data.replace("smalltalk_agent_real","SelfAware")


data.to_csv("intentdata/train.csv",mode="w",index=False)

In [ ]:
# TODO Checks label data balance
import pandas as pd
data = pd.read_csv("intentdata/train.csv") # Original data is a (2000, 7) DataFrame
# data contains 6 feature columns and 1 target column.

# Separate the design matrix from the target labels.
X = data.iloc[:, :-1]
y = data['intent']

y.value_counts().sort_index().plot.bar(x='Target Value', y='Number of Occurrences',figsize=(20,20))

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import json 
import nlpaug.augmenter.word as naw
import nlpaug.flow as naf
print("Loading Models...")
import nltk 
nltk.download('wordnet')
nltk.download('omw-1.4')
TOPK=20 #default=100
ACT = 'insert' #"substitute"
def tst():
    aug_w2v= naw.WordEmbsAug(
        model_type='glove', model_path='glove/glove.6B.300d.txt',
        action="substitute")
    aug_bert = naw.ContextualWordEmbsAug(
        model_path='distilbert-base-uncased', 
        
        action=ACT, top_k=TOPK)
    aug = naf.Sequential([
            aug_bert,aug_w2v
        ])
print("Models Loaded.")
with open("intentdata/intent.json") as f:
    intentwhole = json.load(f)["intents"]
#text = intent[0]["text"][3]
for intent in intentwhole:
    for text in intent["text"]:
        augmented_texts = set()
        for i in range(20):
            aug = naw.SynonymAug(aug_src='wordnet',aug_min=1, aug_max=10, aug_p=i/10)
            augmented_text = str(aug.augment(text))
            print(augmented_text)
            #print(augmented_text)
            augmented_texts.add(augmented_text)
        augmented_texts = augmented_texts.union(set(intent["text"]))
        intent["text"] = list(augmented_texts)
    def test():
        if intent["intent"] != "Jokes": 
            for response in intent["responses"]:
                augmented_responses = set()
                for i in range(20):
                    #aug = naw.SynonymAug(aug_src='wordnet',aug_min=1, aug_max=10, aug_p=i/50,stopwords=["<HUMAN>","<HUMAN>,","<HUMAN>!"])
                    augmented_response = str(aug.augment(response)[0])
                    print(augmented_response)
                    try:
                        augmented_response = augmented_response[:augmented_response.index("<")] + "<HUMAN" + augmented_response[augmented_response.index(">"):]
                    except ValueError as vex:
                        pass
                    #print(augmented_text)
                    augmented_responses.add(augmented_response)
                augmented_responses = augmented_responses.union(set(intent["responses"]))
                intent["responses"] = list(augmented_responses) 
with open("intentdata/intent_aug_text_test.json","w+") as f:
    json.dump({"intents":intentwhole},f)


print(intentwhole[1]["responses"])

In [ ]:
df = pd.read_csv("intentdata/train.csv")
len(list(pd.unique(df["intent"])))

In [ ]:
import os
import json 
import pandas as pd
os.listdir("new_intent_data")

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
nlp("start the fan").similarity(nlp("turn the fan on"))

In [ ]:
import pandas as pd
traindf = pd.read_csv("intentdata/train.csv")
traindf

In [ ]:
import pandas as pd
intentdf = pd.read_csv("new_intent_data/intent_classification.csv")
traindf = pd.read_csv("intentdata/train.csv")
smart_home_intent = intentdf[["text-en","intent-en"]].rename({"text-en":"text","intent-en":"intent"},axis=1)
newdf = pd.concat([traindf,smart_home_intent],axis=0)

newdf.to_csv("intentdata/train.csv",mode="w",index=False)

In [ ]:
with open("new_intent_data/Dataset-train_old.txt") as f:
    textdata = f.readlines()
sent = ""
for text in textdata:
    if text == "":
        pass
    elif text != "":
        if text.count(",") == 2:
            #print(text.replace(",",";",1))
            sent += text.replace(",",";",1)
        else:
            
            sent += text
print(sent)
with open("new_intent_data/Dataset-train.txt","w+") as f:
    f.write(sent)
    

In [ ]:
import pandas as pd
data = pd.read_csv("new_intent_data/Dataset-train.csv",delimiter=",") 
data = data.drop("Unnamed: 0",axis=1)
data
data.to_csv("new_intent_data/Dataset-train.csv",index=False)

In [ ]:
import pandas as pd
data = pd.read_csv("new_intent_data/Dataset-train.csv",delimiter=",") # Original data is a (2000, 7) DataFrame
data = data.apply(lambda x: x.replace('"',''),axis=1)
data.to_csv("new_intent_data/Dataset-train.csv")

In [1]:
import pandas as pd
data = pd.read_csv("new_intent_data/Dataset-train.csv",delimiter="|") 
traindf = pd.read_csv("intentdata/train.csv")
new_data = pd.concat([traindf,data],axis=0)
new_data.to_csv("intentdata/train.csv",index=False)

In [2]:
import pandas as pd
traindf = pd.read_csv("intentdata/train.csv")
traindf

,text,intent
0,listen to westbam alumb allergic on google music,PlayMusic
1,add step to me to the 50 clásicos playlist,AddToPlaylist
2,i give this current textbook a rating value of...,RateBook
3,play the song little robin redbreast,PlayMusic
4,please add iris dement to my playlist this is ...,AddToPlaylist
...,...,...
37306,what is your date of birth,how_old_are_you
37307,what year were you born in,how_old_are_you
37308,what is the year that were you born,how_old_are_you
37309,how old are you ai,how_old_are_you
